In [1]:
import vcf
import argparse
import pandas as pd

In [2]:
def parseVCF(vcffile):
    vcf_variants = {}
    vcf_positions = {}
    pos = []
    var = []
    vcfread = vcf.Reader(open(vcffile))
    samples = vcfread.samples
    sampleName = vcffile
    if len(samples) == 0:
        print("no sample found, consolidating it as known mutations")
        sampleName += "_known_mutations"
    else:
        for sample in samples:
            sampleName += sample
    
    for record in vcfread:
        alts = record.ALT
        chrm = record.CHROM.replace("chr","")
        chrm = chrm.replace("Chr","")
        pos.append(chrm + "_" + str(record.POS))
        vcf_positions[sampleName] = set(pos)

        if "_known_mutations" in sampleName:
            alts = record.ALT
            for alt in alts:
                var.append("_".join([chrm, str(record.POS),record.REF, str(alt)]))
                vcf_variants[sampleName] = set(var)
        
        else:
            for sample in samples:
                alts = record.genotype(sample).gt_bases.split("|")
                if len(alts) == 1:
                    alts = record.genotype(sample).gt_bases.split("/")
                for alt in alts:
                    var.append("_".join([chrm, str(record.POS),record.REF, str(alt)]))
                    vcf_variants[sampleName] = set(var)
                    
                    
    print(vcf_variants)

In [3]:
parseVCF("sample1_final_filtered_indels.vcf")

{'sample1_final_filtered_indels.vcfcancer': {'1_43670069_TTTTG_T', '10_53714808_TTGTG_T', '12_51400686_AATTTT_A', '17_50125403_T_TA', '2_146911686_AG_A', '4_155211731_T_TA', '6_91140045_G_GT', '8_64020897_CTCGGG_C', '9_102041429_A_AC', '9_92087385_A_AC', '15_40940640_AT_AT', '16_10730232_C_CA', '10_54633027_T_TA', '2_159375657_C_CT', '19_52816698_C_CT', '21_39971091_T_TG', '18_38740534_A_AAAG', '11_84772375_CT_C', '6_132775363_GA_G', '6_158744991_C_CT', '1_116654229_AT_AT', '17_13570892_AC_A', '6_92272211_CT_C', '15_100945447_A_AC', '3_35304253_TTGAG_T', '3_110213535_TA_T', '14_76654878_C_CTTTTGCCCA', '4_154893619_GTCA_G', '2_111492471_C_CT', '6_18423997_C_CT', '2_24053168_T_TA', '12_13844496_TC_T', '3_192852136_C_CCA', '4_9040884_A_AC', '6_48456067_CAGTCTTGAAGACAAAATGCTAACATGTAATAAGAAAATATTTGA_C', '2_13928221_G_GA', '5_119909598_GTA_G', '1_246015620_AC_A', '4_78902851_TTAA_T', '12_49333896_G_G', '16_72712738_G_GC', '12_89143004_CAAT_C', '8_108970264_T_TCA', '10_68834956_T_TC', '11_134

In [8]:
def parseCosmic(cosmicVCF):
    vcfreader = vcf.Reader(open(cosmicVCF))
    vcfreader._parse_info
    cosmicMutations = set()
    cosmicMutationScores = dict()
    for record in vcfreader: 
        for alt in record.ALT:
            chrm = record.CHROM.replace('chr', '')
            chrm = chrm.replace('Chr', '')
            identifier = "_".join([chrm, str(record.POS), record.REF, str(alt)])
            cosmicMutations.add(identifier)
            # Fill in fathmm score consolidation here
            # cosmicMutationScores[identifier] = vcfreader._parse_info(record.INFO) 
    return cosmicMutations, cosmicMutationScores